In [ ]:
import sys
!{sys.executable} -m pip install pydrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() 
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=724761270899-po8a50uidu9dikdid7m5klt7eqqcjbb3.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code



In [ ]:
files = {'WRDS_Financial_Ratios_Monthly.zip':'1sR3oAOjKyzIxYttotJrzMmhrShNIV1zf',
         #'WRDS_ALL_PRICE_RETURN_DATA_Daily.zip':'1Fp6lTSOHlEokJsAFtb_MU8onFAN-0Bh8',
         'WRDS_ALL_PRICE_RETURN_DATA_Monthly.zip':'1_ESds4Ede3Zke896hl1d4c3iy8SVR-Yc',
         'WRDS_constiuents1968_2020_output_universe.zip':'1veIbnyv4n3hqBYrQo5ovBoc1aGXdFygc'}

In [ ]:
for file_name, id in files.items():
    print(file_name)
    data = drive.CreateFile({'id': id})
    data.GetContentFile(file_name)

In [ ]:
import shutil
import os
for file_name, id in files.items():
    print(file_name)
    shutil.unpack_archive(file_name, './data')
    os.remove(file_name)

<h1>Start here if you already have the data...</h1>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_filenames = ['.\data\WRDS_Financial_Ratios_Monthly\WRDS_Financial_Ratios_Monthly.csv',
                 #'\data\WRDS_ALL_PRICE_RETURN_DATA_Daily\WRDS_ALL_PRICE_RETURN_DATA_Daily.csv',
                  '.\data\WRDS_ALL_PRICE_RETURN_DATA_Monthly\WRDS_ALL_PRICE_RETURN_DATA_Monthly.csv',
                   '.\data\WRDS_constiuents1968_2020_output_universe\WRDS_constiuents1968_2020_output_universe.csv']

In [ ]:
constituents = pd.read_csv(data_filenames[2], '\t',low_memory=False)

In [ ]:
constituents['DLSTDT'] = np.where(constituents['DLSTDT']==20191231,20201031,constituents['DLSTDT'])
constituents['NAMEDT'] = pd.to_datetime(constituents['NAMEDT'],format="%Y%m%d")
constituents['BEGEXCHDATE'] = pd.to_datetime(constituents['BEGEXCHDATE'],format="%Y%m%d")
constituents['ENDEXCHDATE'] = pd.to_datetime(constituents['ENDEXCHDATE'],format="%Y%m%d")
constituents['DLSTDT'] = pd.to_datetime(constituents['DLSTDT'],format="%Y%m%d")

In [ ]:
date_list = pd.date_range('1968-01-31','2020-10-31', freq='M').tolist()

In [ ]:
temp = pd.DataFrame()
universe = pd.DataFrame(columns=constituents.columns)
for dt in date_list:
    print(dt)
    temp = constituents[(constituents['BEGEXCHDATE'] <= dt) & (constituents['DLSTDT'] >= dt)]
    temp['DATE'] = dt
    print(temp.shape[0])
    universe = universe.append(temp)
    del temp 
    temp = pd.DataFrame()

In [ ]:
universe.shape

In [ ]:
universe.columns

In [ ]:
universe.to_csv('./universe_files/initial_universe.csv',index=False)

In [ ]:
del constituents

<h1>After initial Universe</h1>

In [ ]:
import pandas as pd
import gc
import numpy as np

In [ ]:
universe=pd.read_csv('./universe_files/initial_universe.csv',low_memory=False,dtype={'PERMNO':'string', 'NCUSIP':'string', 'DATE':'string', 'PERMCO':'string','TICKER':'string', 'NCUSIP':'string'})
universe['DATE'] = pd.to_datetime(universe['DATE'],format="%Y-%m-%d")

In [ ]:
date_list = pd.date_range('1968-01-31','2020-10-31', freq='M').tolist()

In [ ]:
financial_ratios = pd.read_csv('.\data\WRDS_Financial_Ratios_Monthly\WRDS_Financial_Ratios_Monthly.csv','\t',low_memory=False,index_col=0,dtype={'gvkey':'string', 'permno':'string', 'adate':'string', 'qdate':'string', 'public_date':'string','TICKER':'string', 'cusip':'string'})

In [ ]:
financial_ratios['public_date'] = pd.to_datetime(financial_ratios['public_date'],format="%Y%m%d")
financial_ratios['divyield'] = financial_ratios['divyield'].str.rstrip('%').astype('float') / 100.0

In [ ]:
universe = pd.merge(universe, financial_ratios, how='left', left_on=['DATE','PERMNO'], right_on = ['public_date','permno'], suffixes=['_PERMNO', '_permno'],copy=False)
universe.to_csv('./universe_files/financial_ratios_universe_1.csv',index=False)
gc.collect()

In [ ]:
universe = pd.merge(universe, financial_ratios,  how='left', left_on=['DATE','NCUSIP'], right_on = ['public_date','cusip'], suffixes=['_NCUSIP', '_cusip'],copy=False)
universe.to_csv('./universe_files/financial_ratios_universe_2.csv',index=False)
gc.collect()

In [ ]:
del financial_ratios

In [ ]:
import pandas as pd
import gc
import numpy as np

date_list = pd.date_range('1968-01-31','2020-10-31', freq='M').tolist()

In [ ]:
universe=pd.read_csv('./universe_files/financial_ratios_universe_2.csv',low_memory=False)
universe['DATE'] = pd.to_datetime(universe['DATE'],format="%Y-%m-%d")

In [ ]:
universe.shape

In [ ]:
summary = pd.DataFrame()
for dt in date_list:
    print(dt)
    temp_df = pd.DataFrame((~universe[universe['DATE']==dt].isna()).sum()).T
    temp_df.index = [dt]
    summary = summary.append(temp_df)
    del temp_df
    gc.collect()
    temp_df = pd.DataFrame()
summary.to_csv('./universe_files/summary_data.csv')

In [ ]:
del summary

In [ ]:
summary=None

In [ ]:
universe = universe[((universe['DATE']>'1969-12-31')&(universe['DATE']<'2020-1-31'))]

In [ ]:
universe = universe.dropna(axis=1, how='all')
universe.to_csv('./universe_files/financial_ratios_universe_3.csv',index=False,low_memory=False)

In [ ]:
universe.shape

In [ ]:
import pandas as pd

In [ ]:
universe=pd.read_csv('./universe_files/financial_ratios_universe_3.csv',low_memory=False)

In [ ]:
universe

In [ ]:
universe.shape

In [ ]:
main_cols = ['DATE','COMNAM','NCUSIP','cusip_NCUSIP','cusip_cusip','PERMNO','permno_cusip','permno_NCUSIP','PERMCO','SICCD','TICKER','TICKER_PERMNO','TICKER_permno']

In [ ]:
second_cols = ['accrual','adv_sale','aftret_eq','aftret_equity','aftret_invcapx','at_turn','bm','CAPEI','capital_ratio','cash_conversion','cash_debt','cash_lt','cash_ratio','cfm','curr_debt','curr_ratio','de_ratio','debt_assets','debt_at','debt_capital','debt_ebitda','debt_invcap','divyield','dltt_be','dpr','efftax','equity_invcap','evm','fcf_ocf','gpm','GProf','int_debt','int_totdebt','intcov','intcov_ratio','inv_turn','invt_act','lt_debt','lt_ppent','npm','ocf_lct','opmad','opmbd','pay_turn','pcf','pe_exi','pe_inc','pe_op_basic','pe_op_dil','PEG_1yrforward','PEG_ltgforward','PEG_trailing','pretret_earnat','pretret_noa','profit_lct','ps','ptb','ptpm','quick_ratio','rd_sale','rect_act','rect_turn','roa','roce','roe','sale_equity','sale_invcap','sale_nwc','short_debt','staff_sale','totdebt_invcap']

In [ ]:
clean_univ = universe[main_cols]

In [ ]:
import numpy as np
import gc

In [ ]:
for col in second_cols:
    clean_univ[col] = np.where(universe[col+'_cusip'].isna(),universe[col+'_NCUSIP'],universe[col+'_cusip'])

In [ ]:
clean_univ.to_csv('./universe_files/financial_ratios_universe_4.csv',index=False)

In [ ]:
summary = pd.DataFrame()
for dt in date_list:
    print(dt)
    temp_df = pd.DataFrame((~clean_univ[clean_univ['DATE']==dt].isna()).sum()).T
    temp_df = temp_df/(temp_df.values.max())
    temp_df.index = [dt]
    summary = summary.append(temp_df)
    del temp_df
    gc.collect()
    temp_df = pd.DataFrame()
summary.to_csv('./universe_files/summary_data_2.csv')

In [ ]:
temp_df = pd.DataFrame((~clean_univ[clean_univ['DATE']==dt].isna()).sum()).T
temp_df = temp_df/(temp_df.values.max())
temp_df.index = [dt]
temp_df

In [ ]:
date_list = pd.date_range('1970-01-31','2019-12-31', freq='M').tolist()
for dt in date_list:
    print(dt)

In [ ]:
clean_univ[clean_univ[second_cols].isna()]

In [ ]:
def unique(list1): 
      
    # insert the list to the set 
    list_set = set(list1) 
    # convert the set to the list 
    unique_list = (list(list_set)) 
    return unique_list

pd.DataFrame(unique(list(clean_univ[clean_univ[second_cols].isna().sum(axis=1)>70]['PERMNO']))).to_csv('permno_codes.txt', index=False)

In [ ]:
clean_univ

In [ ]:
fillna = pd.read_csv('D:\\Project\\e4040-2020Fall-Project-WNZH-jr4001-xy2419-gl2664\\fillna.txt','\t',low_memory=False, dtype={'PERMNO':'string', 'NCUSIP':'string', 'DATE':'string', 'PERMCO':'string','TICKER':'string', 'gvkey':'string'})

In [ ]:
fillna['public_date'] = pd.to_datetime(fillna['public_date'],format="%Y-%m-%d")

In [ ]:
clean_univ = clean_univ.merge(fillna,  how='left', left_on=['DATE','PERMNO'], right_on = ['public_date','permno'], suffixes=['_original', '_copy'],copy=False)

In [ ]:
list(clean_univ.columns)

In [ ]:
main_cols = ['DATE','COMNAM','NCUSIP','cusip_NCUSIP','cusip_cusip','cusip','PERMNO','permno_cusip','permno_NCUSIP','permno','PERMCO','SICCD','TICKER_original','TICKER_PERMNO','TICKER_permno','TICKER_copy']

In [ ]:
second_cols = ['accrual','adv_sale','aftret_eq','aftret_equity','aftret_invcapx','at_turn','bm','CAPEI','capital_ratio','cash_conversion','cash_debt','cash_lt','cash_ratio','cfm','curr_debt','curr_ratio','de_ratio','debt_assets','debt_at','debt_capital','debt_ebitda','debt_invcap','divyield','dltt_be','dpr','efftax','equity_invcap','evm','fcf_ocf','gpm','GProf','int_debt','int_totdebt','intcov','intcov_ratio','inv_turn','invt_act','lt_debt','lt_ppent','npm','ocf_lct','opmad','opmbd','pay_turn','pcf','pe_exi','pe_inc','pe_op_basic','pe_op_dil','PEG_1yrforward','PEG_ltgforward','PEG_trailing','pretret_earnat','pretret_noa','profit_lct','ps','ptb','ptpm','quick_ratio','rd_sale','rect_act','rect_turn','roa','roce','roe','sale_equity','sale_invcap','sale_nwc','short_debt','staff_sale','totdebt_invcap']

In [ ]:
new_clean_univ = clean_univ[main_cols]

In [ ]:
for col in second_cols:
    new_clean_univ[col] = np.where(clean_univ[col+'_original'].isna(),clean_univ[col+'_copy'],clean_univ[col+'_original'])

In [ ]:
new_clean_univ.to_csv('./universe_files/financial_ratios_universe_5.csv',index=False)

In [ ]:
summary = pd.DataFrame()
for dt in date_list:
    print(dt)
    temp_df = pd.DataFrame((~new_clean_univ[new_clean_univ['DATE']==dt].isna()).sum()).T
    temp_df = temp_df/(temp_df.values.max())
    temp_df.index = [dt]
    summary = summary.append(temp_df)
    del temp_df
    gc.collect()
    temp_df = pd.DataFrame()
summary.to_csv('./universe_files/summary_data_2.csv')